In [1]:
import threading
from queue import Queue

In [2]:
threading.active_count()   # 获取已激活的线程数

5

In [3]:
threading.enumerate()    # 查看所有线程信息

[<_MainThread(MainThread, started 16132)>,
 <Thread(Thread-4, started daemon 1344)>,
 <Heartbeat(Thread-5, started daemon 2180)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 14240)>,
 <ParentPollerWindows(Thread-3, started daemon 16684)>]

In [4]:
threading.current_thread()  # 查看正在运行的线程

<_MainThread(MainThread, started 16132)>

# 添加线程
`threading.Thread()`接收参数`target`代表这个线程要完成的任务，需自行定义：

In [6]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

In [7]:
def main():
    thread = threading.Thread(target=thread_job,) # 定义线程
    thread.start()     # 让线程开始工作
if __name__ == '__main__':
    main()

This is a thread of <Thread(Thread-6, started 17368)>


# `join` 功能

In [8]:
import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1) # 任务间隔0.1s
    print("T1 finish\n")

added_thread = threading.Thread(target=thread_job, name='T1')
added_thread.start()
print("all done\n")

T1 start
all done




线程任务还未完成便输出`all done`。如果要遵循顺序，可以在启动线程后对它调用`join`：

In [9]:
import time


def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')


def T2_job():
    print('T2 start\n')
    print('T2 finish\n')


def main():
    added_thread = threading.Thread(target=thread_job, name='T1')
    thread2 = threading.Thread(target=T2_job, name='T2')
    added_thread.start()
    thread2.start()
    thread2.join()
    added_thread.join()

    print('all done\n')


if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

T1 finish

all done



# 储存进程结果 `Queue`

代码实现功能：将数据列表中的数据传入，使用四个线程处理，将结果保存在`Queue`中，线程执行完后，从`Queue`中获取存储的结果。

In [10]:
import threading
import time
from queue import Queue

## 定义一个被多线程调用的函数 
函数的参数是一个列表`l`和一个队列`q`，函数的功能是，对列表的每个元素进行平方计算，将结果保存在队列中：

In [11]:
def job(l, q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)  # 多线程调用的函数不能用 return 返回值

## 定义一个多线程函数 
在多线程函数中定义一个`Queue`，用来保存返回值，代替`return`，定义一个多线程列表，初始化一个多维数据列表，用来处理：

In [16]:
def multithreading():
    q = Queue()  # q中存放返回值，代替return的返回值
    threads = []
    data = [[1, 2, 3], [3, 4, 5], [4, 4, 4], [5, 5, 5]]

在多线程函数中定义四个线程，启动线程，将每个线程添加到多线程的列表中

In [19]:
def multithreading():
    q = Queue()
    threads = []
    data = [[1, 2, 3], [3, 4, 5], [4, 4, 4], [5, 5, 5]]
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i], q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)


if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


# GIL 不一定有效率 

这次我们来看看为什么说 python 的多线程 threading 有时候并不是特别理想. 最主要的原因是就是, Python 的设计上, 有一个必要的环节, 就是 Global Interpreter Lock (GIL). 这个东西让 Python 还是一次性只能处理一个东西.

一段对于 GIL 的解释.
- 尽管Python完全支持多线程编程， 但是解释器的C语言实现部分在完全并行执行时并不是线程安全的。 实际上，解释器被一个全局解释器锁保护着，它确保任何时候都只有一个Python线程执行。 GIL最大的问题就是Python的多线程程序并不能利用多核CPU的优势 （比如一个使用了多个线程的计算密集型程序只会在一个单CPU上面运行）。
- 在讨论普通的GIL之前，有一点要强调的是GIL只会影响到那些严重依赖CPU的程序（比如计算型的）。 如果你的程序大部分只会涉及到I/O，比如网络交互，那么使用多线程就很合适， 因为它们大部分时间都在等待。实际上，你完全可以放心的创建几千个Python线程， 现代操作系统运行这么多线程没有任何压力，没啥可担心的。

In [20]:
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.20099782943725586
1999998000000
multithreading:  0.19199156761169434


# 线程锁 Lock

使用`lock`和不使用`lock`，最后打印输出的结果是不同的。

## 不使用 Lock 的情况 

函数一：全局变量`A`的值每次加`1`，循环10次，并打印。

In [21]:
def job1():
    global A
    for i in range(10):
        A += 1
        print('job1', A)

函数二：全局变量`A`的值每次加`10`，循环`10`次，并打印

In [22]:
def job2():
    global A
    for i in range(10):
        A += 10
        print('job2', A)

主函数：定义两个线程，分别执行函数一和函数二

In [23]:
if __name__ == '__main__':
    A = 0
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2job2
job1 13
job1 14
job1 15
job1 16
job1 17
job1 18
job1  12
job2 29
job2 39
job2 49
job2 59
job2 69
job2 79
job219 89
job2 99
job2 
job1 110
109


可以看出，打印的结果非常混乱。

## 使用 Lock 的情况 
`lock`在不同线程使用同一共享内存时，能够确保线程之间互不影响，使用`lock`的方法是， 在每个线程执行运算修改共享内存之前，执行`lock.acquire()`将共享内存上锁， 确保当前线程执行时，内存不会被其他线程访问，执行运算完毕后，使用`lock.release()`将锁打开， 保证其他的线程可以使用该共享内存。

函数一和函数二加锁

In [24]:
def job1():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 1
        print('job1', A)
    lock.release()


def job2():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 10
        print('job2', A)
    lock.release()

主函数中定义一个 `Lock`：

In [25]:
if __name__== '__main__':
    lock=threading.Lock()
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110
